In [22]:
from pymongo import MongoClient
import pandas as pd
import urllib.parse

# Credentials
username = urllib.parse.quote_plus("soundDev")
password = urllib.parse.quote_plus("e1kemyOwhAYXNidM")
cluster_url = "atlas-sql-681e2eb821f0c21b8b5e1712-lkspta.a.query.mongodb.net"
mongo_uri = f"mongodb://{username}:{password}@{cluster_url}/checkoutETL?ssl=true&authSource=admin"

# Connect
client = MongoClient(mongo_uri)
collection = client["checkoutETL"]["transactions"]

# Only CIT, SALE, Non-PayPal merchants
query_filter = {
    "billingCycleNumber": 1,
    "txnType": "SALE",
    "merchant": {"$nin": ["Paypal US", "Paypal USD (Josh)", "PayPal@OM"]}
}

# Only required fields
projection = {
    'transactionId': 1,
    'orderId': 1,
    'dateCreated': 1,
    'billingCycleNumber': 1,
    'campaignName': 1,
    'merchant': 1,
    'cardBin': 1,
    'totalAmount': 1,
    'responseType': 1,
    'txnType': 1,
    'chargebackAmount': 1,
    '_id': 0
}

# Pull data
df = pd.DataFrame(list(collection.find(query_filter, projection)))


In [24]:
df.shape
 

(78294, 11)

In [25]:
import numpy as np

# 1. Standardize response type values
df['responseType_clean'] = df['responseType'].str.strip().str.upper()

# 2. Create a key based on orderId + totalAmount
df['match_key'] = df['orderId'].astype(str) + '|' + df['totalAmount'].astype(str)

# 3. Identify which orderId+amount combos had a SUCCESS
success_keys = set(df.loc[df['responseType_clean'] == 'SUCCESS', 'match_key'])

# 4. Use np.where (vectorized) to keep:
# - All SUCCESS rows
# - All others only if no matching SUCCESS exists
df['Keep'] = np.where(
    (df['responseType_clean'] == 'SUCCESS') |
    (~df['match_key'].isin(success_keys)),
    'Include',
    'Exclude'
)

# 5. Final filtered DataFrame
df_final = df[df['Keep'] == 'Include'].drop(columns=['responseType_clean', 'match_key', 'Keep'])


In [27]:
df_final.shape

(72002, 11)

In [28]:
print(df_final.head())

   transactionId  billingCycleNumber                       campaignName  \
0         438550                   1                     MozzGuard - CA   
1         438554                   1                     MozzGuard - CA   
2         438591                   1                   Blaze Buddy - AU   
3         438607                   1                           OptiFuel   
4         438657                   1  BTE Hearing CIC Pro (Native) - UK   

  cardBin chargebackAmount         dateCreated           merchant     orderId  \
0  522303             None 2025-05-02 00:53:02     EMS Josh (NMI)  C67E30D090   
1  543446             None 2025-05-02 00:57:39     EMS Josh (NMI)  C7FEB1B9E8   
2  521729             None 2025-05-02 01:32:39           Adyen US  090E3EA41B   
3  445220             None 2025-05-02 01:46:44  Payarc Josh (NMI)  D713397623   
4  492913             None 2025-05-02 02:28:05           ADYEN AU  8434DCBAF3   

  responseType  totalAmount txnType  
0      SUCCESS        89

In [29]:
df_final['merchant'].unique()

array(['EMS Josh (NMI)', 'Adyen US', 'Payarc Josh (NMI)', 'ADYEN AU',
       'Quantum (NMI)', 'Seamless Chex (Auth.net)', 'Phoenix (NMI)',
       'Quantum PremHealth (NMI)', 'Finix Josh (Auth.net)',
       'Airwallex (AU)', 'NMI (EMS)', 'Stripe (Josh)', None], dtype=object)

In [31]:
df_final['txnType'].unique()

array(['SALE'], dtype=object)

In [32]:
df_final['billingCycleNumber'].unique()

array([1], dtype=int64)

In [34]:
# STEP 1: Clean responseType
df_final['responseType_clean'] = df_final['responseType'].str.strip().str.upper()

# STEP 2: Flagging transaction outcomes
df_final['is_success'] = df_final['responseType_clean'] == 'SUCCESS'
df_final['is_hard'] = df_final['responseType_clean'] == 'HARD_DECLINE'
df_final['is_soft'] = df_final['responseType_clean'] == 'SOFT_DECLINE'


In [35]:
grouped = df_final.groupby(['campaignName', 'merchant']).agg(
    total_txns=('transactionId', 'count'),
    success_count=('is_success', 'sum'),
    hard_declines=('is_hard', 'sum'),
    soft_declines=('is_soft', 'sum')
).reset_index()


In [36]:
top_10_campaigns = (
    grouped.groupby('campaignName')['total_txns']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .index.tolist()
)

grouped = grouped[grouped['campaignName'].isin(top_10_campaigns)]


In [37]:
# Filter merchants with at least 20 successes
grouped = grouped[grouped['success_count'] > 20]

# Add success rate and decline %
grouped['auth_rate'] = (grouped['success_count'] / grouped['total_txns'] * 100).round(2)
grouped['%_hard'] = (grouped['hard_declines'] / grouped['total_txns'] * 100).round(2)
grouped['%_soft'] = (grouped['soft_declines'] / grouped['total_txns'] * 100).round(2)


In [38]:
# Campaign totals for success and txn count
campaign_sums = grouped.groupby('campaignName').agg(
    total_txns_campaign=('total_txns', 'sum'),
    total_success_campaign=('success_count', 'sum')
).reset_index()

# Merge into grouped data
grouped = grouped.merge(campaign_sums, on='campaignName', how='left')

# Routing %s
grouped['current_routing_percentage'] = (grouped['total_txns'] / grouped['total_txns_campaign'] * 100).round(2)
grouped['recommended_routing_percentage'] = (grouped['success_count'] / grouped['total_success_campaign'] * 100).round(2)


In [39]:
grouped['merchant_rank'] = grouped.groupby('campaignName')['success_count'].rank(method='first', ascending=False)
final = grouped[grouped['merchant_rank'] <= 4].copy()


In [40]:
final_output = final[[
    'campaignName', 'merchant', 'total_txns', 'success_count', 'auth_rate',
    '%_hard', '%_soft', 'current_routing_percentage', 'recommended_routing_percentage'
]].rename(columns={
    'campaignName': 'campaign',
    'merchant': 'gateway'
}).sort_values(['campaign', 'success_count'], ascending=[True, False])


In [47]:
final_output


,campaign,gateway,total_txns,success_count,auth_rate,%_hard,%_soft,current_routing_percentage,recommended_routing_percentage
6,BTE Hearing CIC Pro (Facebook),Stripe (Josh),2752,1830,66.50,0.00,33.50,50.25,55.44
0,BTE Hearing CIC Pro (Facebook),Adyen US,1264,888,70.25,0.55,29.19,23.08,26.90
1,BTE Hearing CIC Pro (Facebook),EMS Josh (NMI),398,219,55.03,22.11,22.86,7.27,6.63
4,BTE Hearing CIC Pro (Facebook),Quantum PremHealth (NMI),394,172,43.65,38.83,17.51,7.19,5.21
11,BTE Hearing CIC Pro (Native),Stripe (Josh),767,543,70.80,0.00,29.20,57.63,66.87
9,BTE Hearing CIC Pro (Native),Quantum (NMI),251,126,50.20,27.89,21.91,18.86,15.52
10,BTE Hearing CIC Pro (Native),Seamless Chex (Auth.net),211,88,41.71,3.32,54.98,15.85,10.84
8,BTE Hearing CIC Pro (Native),EMS Josh (NMI),37,28,75.68,13.51,10.81,2.78,3.45
12,BTE Hearing CIC Pro (Native) - AU,ADYEN AU,1750,1374,78.51,0.00,21.49,100.00,100.00
14,BTE Hearing CIC Pro (Native) - UK,Adyen US,4312,2406,55.80,0.35,43.85,58.29,63.38
